# Checkpointing DataFrames

Sometimes execution plans can get pretty long and Spark might run into trouble. Common scenarios are iterative algorithms like ML or graph algorithms, which contain a big outer loop and iteratively transform a DataFrame over and over again. This would result in a really huge execution plan.

In these cases you could use `cache()` or `persist()` in order to improve performance (otherwise all steps of the loop would be executed again from the very beginning leading to a runtime of O(n^2)). But this will not cut off the lineage.

Checkpointing is the right solution for these cases. It will persist the data of a DataFrame in a reliable distributed storage (most commonly HDFS) and cut off the lineage.

# 1 Load Data

We will load the weather data again for this example.

In [ ]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

In [ ]:
from pyspark.sql.functions import *
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

Use a single year to keep execution plans small

In [ ]:
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))

In [ ]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)
    
weather.limit(10).toPandas()

## 1.2 Load Station Metadata

In [ ]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# Display first 10 records    
stations.limit(10).toPandas()

# 2 Join Data

Now we perform the join between the station master data and the measurements, as we did before.

In [ ]:
joined_weather = weather.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))

# 3 Truncating Execution Plans

Now we want to understand the effect of checkpointing. First we will use the traditional aggregation and print the execution plan.

## 3.1 Traditional Aggregation

In [ ]:
result = joined_weather.groupBy(joined_weather["ctry"], joined_weather["year"]).agg(
        min(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('min_temp'),
        max(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('max_temp')
)

# YOUR CODE HERE

## 3.2 Checkpointing

Now we first checkpoint the joined weather data set and then perform the aggregation on the checkpointed DataFrame.

### Set Checkpoint directory

First we need to specify a checkpoint directory on a reliable shared file system.

In [ ]:
# YOUR CODE HERE

### Create checkpoint

Now we can create a checkpoint for the joined weather. Note that this takes some time, as checkpointing is not a lazy operation, it will be executed immediately. This is also conceptionally neccessary, because one aspect of checkpointing is that the whole lineage gets cut off. So there is no way around executing the computation for materializing the DataFrame inside the checkpoint directory

In [ ]:
# YOUR CODE HERE

### Inspect execution plan

Let us have a look at the execution plan of the checkpointed DataFrame

In [ ]:
# YOUR CODE HERE

As you can see, the lineage got lost and is replaced by a `Scan ExistingRDD` which refers to the data in the checkpoint directory.

### Perform aggregation

Now we can perform the aggregation with the checkpointed variant of the joined weather DataFrame.

In [ ]:
result = cp_weather.groupBy(cp_weather["ctry"], cp_weather["year"]).agg(
        min(when(cp_weather.air_temperature_qual == lit(1), cp_weather.air_temperature)).alias('min_temp'),
        max(when(cp_weather.air_temperature_qual == lit(1), cp_weather.air_temperature)).alias('max_temp')
)

# YOUR CODE HERE

As expected, the execution plan now essentially only contains the aggregation in three steps (partial aggregation, shuffle, final aggregation). The lineage of the join is not present any more.